In [ ]:
import os
import shutil
!apt -y update -qq
!apt -y install -qq aria2
%env TF_CPP_MIN_LOG_LEVEL=1
%env UDA_VISIBLE_DEVICES=0,1

#!pip uninstall torch torchversion -y
!pip install -q torch==1.13.1+cu117 torchvision==0.14.1+cu117 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl
!pip install -q xformers==0.0.16
!pip install -q huggingface-hub==0.11.0 -U

!git clone -b v2.0 https://github.com/camenduru/stable-diffusion-webui
!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O /kaggle/working/stable-diffusion-webui/scripts/run_n_times.py
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser /kaggle/working/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
!git clone https://github.com/camenduru/stable-diffusion-webui-huggingface /kaggle/working/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface
!git clone -b v2.0 https://github.com/camenduru/sd-civitai-browser /kaggle/working/stable-diffusion-webui/extensions/sd-civitai-browser
!git clone https://github.com/kohya-ss/sd-webui-additional-networks /kaggle/working/stable-diffusion-webui/extensions/sd-webui-additional-networks
!git clone https://github.com/etherealxx/batchlinks-webui /kaggle/working/stable-diffusion-webui/extensions/batchlinks-webui
%cd /kaggle/working/stable-diffusion-webui

!sed -i -e 's/numpy==1.23.3/numpy==1.21.6/g' requirements_versions.txt
!sed -i -e 's/blendmodes==2022/blendmodes==2021/g' requirements_versions.txt
!sed -i -e 's/fastapi==0.90.1/fastapi==0.89.1/g' requirements_versions.txt
!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /kaggle/working/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /kaggle/working/stable-diffusion-webui/launch.py

!npm install -g localtunnel
import subprocess
import threading
import time
import socket
def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
    for line in p.stdout:
        print(line.decode(), end='')
threading.Thread(target=iframe_thread, daemon=True, args=(7860,)).start()

#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/anything-v4.5-vae-swapped/resolve/main/anything-v4.5-vae-swapped.safetensors -d /kaggle/working/stable-diffusion-webui/models/Stable-diffusion -o anything-v4.5-vae-swapped.safetensors

# 删除原始模型
#os.remove('/kaggle/working/stable-diffusion-webui/models/Stable-diffusion/anything-v4.5-vae-swapped.safetensors')
#os.mkdir('/kaggle/working/stable-diffusion-webui/models/Lora')

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/94640 -d /kaggle/working/stable-diffusion-webui/models/Stable-diffusion -o majicmixRealistic_v6.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d /kaggle/working/stable-diffusion-webui/models/Stable-diffusion -o majicmixRealistic_v4.vae.pt

# ----- LORA -----
# https://civitai.com/models/100371
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/107433 -d /kaggle/working/stable-diffusion-webui/models/Lora -o kuriyama_mirai_v1.safetensors
# https://civitai.com/models/58390/detail-tweaker-lora-lora
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/62833 -d /kaggle/working/stable-diffusion-webui/models/Lora -o add_detail.safetensors
# https://civitai.com/models/7808/easynegative
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/9208 -d /kaggle/working/stable-diffusion-webui/models/Lora -o EasyNegative.safetensors
# https://civitai.com/models/5224/bad-artist-negative-embedding
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/6056 -d /kaggle/working/stable-diffusion-webui/models/Lora -o Bad_artist_Negative.pt
# https://civitai.com/models/89348
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/95103 -d /kaggle/working/stable-diffusion-webui/models/Lora -o ChineseStyle_v1.safetensors

# # 下载ChilloutMix模型
# !wget https://civitai.com/api/download/models/11745
# # 重命名文件
# os.rename("/kaggle/working/stable-diffusion-webui/11745","/kaggle/working/stable-diffusion-webui/ChilloutMix.safetensors")
# # 移动至主模型文件夹
# shutil.move("/kaggle/working/stable-diffusion-webui/ChilloutMix.safetensors",'/kaggle/working/stable-diffusion-webui/models/Stable-diffusion')

# # 下载Counterfeit模型
# !wget https://civitai.com/api/download/models/57618
# # 重命名文件
# os.rename("/kaggle/working/stable-diffusion-webui/57618","/kaggle/working/stable-diffusion-webui/Counterfeit.safetensors")
# # 移动至主模型文件夹
# shutil.move("/kaggle/working/stable-diffusion-webui/Counterfeit.safetensors",'/kaggle/working/stable-diffusion-webui/models/Stable-diffusion')

# # 下载BRA模型
# !wget https://civitai.com/api/download/models/63786
# # 重命名文件
# os.rename("/kaggle/working/stable-diffusion-webui/63786","/kaggle/working/stable-diffusion-webui/BRA.safetensors")
# # 移动至主模型文件夹
# shutil.move("/kaggle/working/stable-diffusion-webui/BRA.safetensors",'/kaggle/working/stable-diffusion-webui/models/Stable-diffusion')

# # 下载BeautyProMix模型
# !wget https://civitai.com/api/download/models/19597
# # 重命名文件
# os.rename("/kaggle/working/stable-diffusion-webui/19597","/kaggle/working/stable-diffusion-webui/BeautyProMix.safetensors")
# # 移动至主模型文件夹
# shutil.move("/kaggle/working/stable-diffusion-webui/BeautyProMix.safetensors",'/kaggle/working/stable-diffusion-webui/models/Stable-diffusion')

# # 下载GuoFeng3模型
# !wget https://civitai.com/api/download/models/36644
# # 重命名文件
# os.rename("/kaggle/working/stable-diffusion-webui/36644","/kaggle/working/stable-diffusion-webui/GuoFeng3.safetensors")
# # 移动至主模型文件夹
# shutil.move("/kaggle/working/stable-diffusion-webui/GuoFeng3.safetensors",'/kaggle/working/stable-diffusion-webui/models/Stable-diffusion')

# # 下载KoreanDollLikeness
# !wget https://civitai.com/api/download/models/31284
# # 重命名文件
# os.rename("/kaggle/working/stable-diffusion-webui/31284","/kaggle/working/stable-diffusion-webui/KoreanDollLikeness.safetensors")
# # 移动至主模型文件夹
# shutil.move("/kaggle/working/stable-diffusion-webui/KoreanDollLikeness.safetensors",'/kaggle/working/stable-diffusion-webui/models/Lora')

# # 下载KoreanDollLikeness_v10
# !wget https://civitai.com/api/download/models/22968
# # 重命名文件
# os.rename("/kaggle/working/stable-diffusion-webui/22968","/kaggle/working/stable-diffusion-webui/koreanDollLikeness_v10.safetensors")
# # 移动至主模型文件夹
# shutil.move("/kaggle/working/stable-diffusion-webui/koreanDollLikeness_v10.safetensors",'/kaggle/working/stable-diffusion-webui/models/Lora')

!python launch.py --share --xformers --enable-insecure-extension-access --theme dark --gradio-queue

219 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: An error occurred during the signature verification. The repository is not updated and the previous index files will be used. GPG error: http://packages.cloud.google.com/apt gcsfuse-focal InRelease: The following signatures couldn't be verified because the public key is not available: NO_PUBKEY B53DC80D13EDEF05
W: An error occurred during the signature verification. The repository is not updated and the previous index files will be used. GPG error: https://packages.cloud.google.com/apt cloud-sdk InRelease: The following signatures couldn't be verified because the public key is not available: NO_PUBKEY B53DC80D13EDEF05
W: An error occurred during the signature verification. The repository is not updated and the previous index files will be used. GPG error: https://packages.cloud.google.com/apt google-fast-socket InRelease: The following signatures couldn't be verified because the public key is not available: NO_PU